# Compatible embeddings using Allombert's algorithm

In the following we represent $K\otimes C$ as $K[t]/\Phi(t)$ where $\Phi(t)$ is the polynomial defining 
the field $C = k(\zeta)$.

In [208]:
"""
Compute the frobenius σ' of `elem` in K⊗C defined by σ'(x⊗y) = σ(x)⊗y with 
σ the frobenius of K/k with k the prime subfield.
"""
def froby(elem):
    l = elem.list()
    f = parent(l[0]).frobenius_endomorphism()
    res = parent(elem)()
    t = parent(elem).gen()
    for j in range(len(l)):
        res += f(l[j])*t^j
    return res  

"""
Compute the matrix representing the frobenius σ' in K⊗C, over the basis B⊗1
where B is a basis of K over k. The coefficients are in C=k(ζ).
"""
def frob_mat(K, C = None):
    
    if C == None:
        C = K.prime_subfield()
    
    frob = K.frobenius_endomorphism()
    n = K.degree()
    S = MatrixSpace(C, n)
    g = K.gen()
    
    cols = []
    
    for j in range(n):
        s = frob(g^j)
        l = s.polynomial().list()
        l = l + (n-len(l))*[C()]
        cols.append(l)
        
    return S(cols).transpose()

"""
Compute a solution of the equation ``σ'(x) = (1⊗ζ)x`` in K⊗C = T.
"""
def hilbert90(K, C, T):
    
    M = frob_mat(K, C)
    t = C.gen()
    eigen = M.right_eigenvectors()
    
    for e in eigen:
        if e[0] == t:
            v = e[1][0]
            break
    
    res = T()
    g = K.gen()
    for j in range(K.degree()):
        res += T(v[j])*g^j
        
    return res

"""
Compute the first coordinate of `elem` in `basis`, knowing that
`elem` is in fact in a subspace of degree `d2`.
"""
def change_basis(elem, basis, d2):
    
    k = elem.base_ring()
    d1 = elem.parent().degree()
    S = MatrixSpace(k, d1, d2)
    A = S()
    
    for i in range(d2):
        L = (basis^i).list()
        for j in range(d1):
            A[j,i] = L[j]
            
    S2 = MatrixSpace(k, d1, 1)
    B = S2(elem.list())
    X = A.solve_right(B)
    
    return X[0,0]


"""
Compute two elements β and γ such that k1 = k(β) can be embedded
in k2 via the map β |-> γ. 
"""
def allombert_gens(k1, k2):
    
    # We first set some usefull variables
    k = k1.prime_subfield()
    R = PolynomialRing(k, "x")
    Rxy.<x,y> = PolynomialRing(k, "x, y")
    m, n = k1.degree(), k2.degree()
    
    # Then we create the spaces necessary to solve Hilbert 90
    # C1 = k(ζ1), T1 = k1 ⊗ C1
    # and we solve Hilbert 90 in T1
    f1 = factor(R(cyclotomic_polynomial(m)))[0][0]
    C1 = k.extension(f1, "t")
    T1 = k1.extension(f1, "t")
    a1 = hilbert90(k1, C1, T1)
    
    # After that we find a compatible polynomial f2 defining our root of unity ζ2
    f1xy = Rxy(f1)
    res = f1xy.resultant(y^(n/m)-x, x)
    P = gcd(R(res.polynomial(y).list()), R(cyclotomic_polynomial(n)))
    f2 = factor(P)[0][0]
    
    # We create the spaces relative to k2 and ζ2
    # and solve Hilbert 90 in T2 = k2 ⊗ k(ζ2)
    C2 = k.extension(f2, "t")
    T2 = k2.extension(f2, "t")
    a2 = hilbert90(k2, C2, T2)
    
    
    # We then coerce a1^m, which lives in T1, in C2 = k(ζ2) using
    # the embedding ζ1 |-> ζ2^(n/m) to see k(ζ1) in k(ζ2)
    b1 = C2([k(x) for x in (a1^m).list()]).polynomial().subs(C2.gen()^(n/m))
    
    # We coerce a2^n, which lives in T2, in C2 = k(ζ2)
    b2 = C2([k(x) for x in (a2^n).list()])
    
    # We compute an m-th root of b1/b2 = a1^m/a2^n
    c = (b1*b2^(-1)).nth_root(m)
    
    # And we finally return the first coefficient of a1 and c × a2 in
    # the base (1 ⊗ ζ2) PROBLEM HERE ?? 
    
    # --! PROBLEM !--
    # Maybe we are computing the first coefficient of c × a2 in the basis 
    # (1 ⊗ ζ2) and that should be (1 ⊗ ζ1) = (1 ⊗ ζ2^(n/m)) ???
    return a1.list()[0], change_basis(T2(c)*a2^(n/m), T2.gen()^(n/m), C1.degree())

In [209]:
p = 5
k = GF(p)
k12 = GF(p^12)
k24 = GF(p^24)
k48 = GF(p^48)

R = PolynomialRing(k, "x")
Rxy.<x, y> = PolynomialRing(k, "x, y")

In [237]:
a, b = allombert_gens(k12, k48)
a.minpoly() == b.minpoly()

True

# Tests